# Examining how geospatial data can help improve predicting housing prices
Group 9 - Julius Løve Fischer, Hans Chirstian Jul Lehmann & Kerem Yapici

## Importing packages and unzipping data from boliga.dk

In [1]:
import numpy as np
import requests
from datetime import date
import time
import math
import os
import ast
import json
import matplotlib.pyplot as plt
import pandas as pd
import tqdm
import seaborn as sns
import folium
import geopandas as gpd
import fiona
from shapely.geometry import Point
%matplotlib inline

In [2]:
houses = pd.read_csv('houses.csv')

column_selection = ['address', 'zipCode', 'price', 'soldDate', 'propertyType',
            'saleType', 'sqmPrice', 'rooms', 'size', 'buildYear', 'guid', 'latitude',
            'longitude', 'municipalityCode', 'city']

houses = houses[column_selection].query("saleType == 'Alm. Salg'") # select relevant columns and sale types
houses['unitId'] = houses['guid'].str.lower() # Convert to lower case for merging with bbr dataframe
houses['soldDate'] = pd.to_datetime(houses['soldDate'].astype(str)).dt.date
houses = houses.sort_values(by="soldDate", ascending = False) # Dataframe is not sorted by default as we have two query_lists
houses = houses.set_index('soldDate').drop('guid', axis=1)

houses

,address,zipCode,price,propertyType,saleType,sqmPrice,rooms,size,buildYear,latitude,longitude,municipalityCode,city,unitId
soldDate,,,,,,,,,,,,,,
2022-07-27,"Gyldenlakvej 27, 1. tv",2300,2340000,3,Alm. Salg,41052.633,2.0,57,1938,55.645130,12.612886,101,København S,f08ba307-5131-485f-8a9c-a51b53511aa3
2022-07-27,"Gammel Kongevej 165A, 1",1850,5995000,3,Alm. Salg,66611.110,3.0,90,2000,55.677700,12.535549,147,Frederiksberg C,5ddda144-2d9e-413d-9cbe-468522500a68
2022-07-27,"Rolfsvej 37, 2. th",2000,7887000,3,Alm. Salg,71054.055,4.0,111,1916,55.683937,12.531795,147,Frederiksberg,cf5a390d-4b0b-4f11-a79f-109e540864e0
2022-07-26,"Havneholmen 82, 2. th",1561,5200000,3,Alm. Salg,68421.055,2.0,76,2008,55.661070,12.564086,101,København V,31c9cf15-56cd-4e0c-8324-5e9f34bfd78b
2022-07-22,"Islands Brygge 24B, 4. tv",2300,7825000,3,Alm. Salg,74523.810,3.0,105,1910,55.663128,12.571215,101,København S,95e09d1f-dcf2-4e5c-abb4-fd3e753a7c3c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2000-01-02,"Lykkesholms Allé 10, 2. th",1902,1850000,3,Alm. Salg,22289.156,4.0,83,1876,55.676643,12.548173,147,Frederiksberg C,5e3fd65e-27a7-46df-9893-03284d76c75b
2000-01-02,"Lykkesholms Allé 10, 1",1902,1850000,3,Alm. Salg,14015.151,4.0,132,1876,55.676643,12.548173,147,Frederiksberg C,3df49d93-7b12-44bd-a75a-d6e61ed6972e
2000-01-02,"Lykkesholms Allé 10, 2. tv",1902,1850000,3,Alm. Salg,108823.530,1.0,17,1876,55.676643,12.548173,147,Frederiksberg C,1a503662-4e1b-4e37-a991-85e3f01789fc


In [4]:
# Reading the gzip file
bbr = pd.read_parquet('df.parquet.gzip')
unpack_values = ['bbrInfoBox', 'buildingInfo', 'unitInfo']
for i in unpack_values:
    bbr = pd.concat([bbr, bbr[i].apply(ast.literal_eval).apply(pd.Series)], axis=1)
bbr = bbr.drop(unpack_values, axis=1)

In [5]:
####### GEM EVT. DEN HER FIL LOKALT SOM CSV-FIL, SÅ OG READ DEN, SÅ I IKKE SKAL KØRE HELE NOTEBOOKEN ######
boliga = pd.merge(houses, bbr, on='unitId', how='inner') # Inner merge to discard NaN's, if any

In [7]:
# Saving the merged file - will use this from now on
path = r'C:\Users\Kerem\Desktop'

boliga.to_csv(path +'/boliga.csv')

## Data cleaning

In [37]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

#This way, we can count the number of missing values per column;
boliga.isnull().sum() 

#addressFloor has 14417 missings

# only drop rows where all columns are NaN
boliga.dropna(how='all')

In [34]:
boliga.columns

Index(['address', 'zipCode', 'price', 'propertyType', 'saleType', 'sqmPrice',
       'rooms', 'size', 'buildYear', 'latitude', 'longitude',
       'municipalityCode', 'city', 'unitId', 'addressFloor',
       'ejendomsværdiskat', 'grundskyld', 'propertyTypeName',
       'monthlyPropertyTax', 'lotSize', 'roomsQuantity', 'area',
       'areaRecorded', 'evaluationPrice', 'basementSize', 'buildingNumber',
       'buildingUsage', 'constructionYear', 'floorQuantity',
       'latesstRebuildYear', 'buildingArea', 'businessArea', 'livingArea',
       'conservationCode', 'conservationWorthy', 'heatingSupplyCode',
       'outerWalls', 'roof', 'buildInGarageArea', 'buildInCarportArea',
       'buildInOuthouseArea', 'conservatoryArea', 'coveredArea',
       'openCoveredArea', 'closedCoveredArea', 'carbageRoomArea', 'otherArea',
       'accessAreaArea', 'waterSupplyCode', 'drainCode', 'drainPermission',
       'heatingCode', 'additionalHeatingSupplyCode',
       'additionalOuterWallMaterial', 'additi